In [1]:
import helium
import pandas as pd
import os
import time
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [2]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

In [3]:
time.sleep(10)
#Required to activate card-section below
browser.maximize_window()
lt=[t for t in browser.find_elements_by_class_name('card-section') 
   if t.text.find('Latest Activity')>-1][0]

In [4]:
lt.find_element_by_tag_name('a').click()

Web page with latest activity

General info

In [5]:
time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

Load tables

In [6]:
browser.find_element_by_id('show-hidden-efforts').click()

In [7]:
html=browser.page_source

In [8]:
ll=pd.read_html(html)

General info table

In [9]:
mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

In [10]:
mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

In [11]:
mtdt

{'Calories': {'Avg': '590', 'Max': '590'},
 'Elapsed Time': {'Avg': '59:50', 'Max': '59:50'},
 'Speed': {'Avg': '8.8km/h', 'Max': '59.8km/h'},
 'date': '2:31 PM on Monday, May 25, 2020',
 'distance': '8.56km',
 'elevation': '468m',
 'energy': '530kJ',
 'name': 'La Tuna',
 'power': '468m',
 'time': '58:07'}

Segments

In [12]:
sgmt=ll[1][['Name','Time','Speed','Power']]

In [13]:
sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

In [14]:
sgmt['Info']=''

In [15]:
sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [16]:
sgmt[:1]

,Name,Time,Speed,Power,Info
0,"Loma linda, detrás del antiguo principito 0.4...",2:11,12.7km/h,215W,"{'energy': '530kJ', 'power': '468m', 'date': '..."


In [17]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [18]:
df

,Info,Name,Power,Speed,Time
0,"{'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:...","Loma linda, detrás del antiguo principito 0.4...",206W,11.4km/h,2:26
1,"{'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:...",La Vaquita - Torre Eiffel (Sabaneta) 0.46km ...,183W,5.8km/h,4:47
2,"{'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:...",La doctora - Lomitas 0.33km 41m 12%,223W,5.5km/h,3:39
3,"{'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:...",Lomita Aves María - Espigas 0.52km 52m 10%,214W,6.5km/h,4:50
4,"{'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:...",lomita avesmaría 0.16km 15m 10%,273W,7.6km/h,1:16
5,"{'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:...",Calle 56 S Climb 2.18km 362m 17%,170W,3.5km/h,37:39
6,"{'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:...",San Judas Completo hasta Alto (La Tuna) 2.29k...,171W,3.7km/h,37:08
7,"{'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:...",Calle 56 S Climb 2.08km 344m 16%,171W,3.4km/h,37:00
8,"{'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:...",Loma de San Judas 2.23km 358m 15%,179W,3.5km/h,38:33
9,"{'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:...",San Judas (Desde Puente hasta caseta de Don Fa...,171W,3.2km/h,33:00


In [23]:
df.to_json(db_file)

In [33]:
f=open('commit.sh','w')
f.write('commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [ ]:
!bash ./commit.sh

In [22]:
df.loc[1,'Info']

{'Calories': {'Avg': '611', 'Max': '611'},
 'Elapsed Time': {'Avg': '1:08:13', 'Max': '1:08:13'},
 'Speed': {'Avg': '7.8km/h', 'Max': '57.6km/h'},
 'date': '1:49 PM on Sunday, May 24, 2020',
 'distance': '8.20km',
 'elevation': '447m',
 'energy': '548kJ',
 'name': 'La Tuna',
 'power': '447m',
 'time': '1:03:18'}